In [1]:
# If this notebook is in a subdirectory of the project's root,
# update the PYTHONPATH to be able to import as usual
import os, sys
project_root = os.path.dirname(os.path.abspath(''))
sys.path.append(project_root)

In [2]:
# Standard Imports
import torch
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

# Project Imports
from train import setup
from utils import save_path

In [6]:
hs = np.array([0.8, 0.4, 0.2, 0.1, 0.05])
methods = ['euler-forw', 'euler-symp']

## Outsource the generation of a single plot

In [9]:
def plot_loss(axes, args):
    for method in methods:
        for h in hs:
            # Setup desired args
            args = setup(args | {'loss_type': method, 'h': h}, save_dir_prefix='/results/experiment-')

            # Load loss log
            saved_dict = torch.load(save_path(args), map_location=torch.device('cpu'))
            train_loss, test_loss = saved_dict['stats']
            
            axes.plot(train_loss)
            axes.plot(test_loss)

## Begin Plotting Code

In [10]:
from visualize import *
mpl.rcParams.update(params)

In [12]:
# TESTING: The below needs all methods for one problem
fig, axes = plt.subplots()
plot_loss(axes, default_args | {'name': 'pendulum'})
plt.show()

KeyError: 'stats'